In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

path = '/kaggle/input/tabular-playground-series-mar-2022'

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-mar-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-mar-2022/train.csv
/kaggle/input/tabular-playground-series-mar-2022/test.csv


In [2]:
train = pd.read_csv(f'{path}/train.csv', parse_dates=['time']).drop(columns=['row_id'])
train

,time,x,y,direction,congestion
0,1991-04-01 00:00:00,0,0,EB,70
1,1991-04-01 00:00:00,0,0,NB,49
2,1991-04-01 00:00:00,0,0,SB,24
3,1991-04-01 00:00:00,0,1,EB,18
4,1991-04-01 00:00:00,0,1,NB,60
...,...,...,...,...,...
848830,1991-09-30 11:40:00,2,3,NB,54
848831,1991-09-30 11:40:00,2,3,NE,28
848832,1991-09-30 11:40:00,2,3,SB,68
848833,1991-09-30 11:40:00,2,3,SW,17


In [3]:
test = pd.read_csv(f'{path}/test.csv', parse_dates=['time']).drop(columns=['row_id'])
test

,time,x,y,direction
0,1991-09-30 12:00:00,0,0,EB
1,1991-09-30 12:00:00,0,0,NB
2,1991-09-30 12:00:00,0,0,SB
3,1991-09-30 12:00:00,0,1,EB
4,1991-09-30 12:00:00,0,1,NB
...,...,...,...,...
2335,1991-09-30 23:40:00,2,3,NB
2336,1991-09-30 23:40:00,2,3,NE
2337,1991-09-30 23:40:00,2,3,SB
2338,1991-09-30 23:40:00,2,3,SW


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 848835 entries, 0 to 848834
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   time        848835 non-null  datetime64[ns]
 1   x           848835 non-null  int64         
 2   y           848835 non-null  int64         
 3   direction   848835 non-null  object        
 4   congestion  848835 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 32.4+ MB


In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2340 entries, 0 to 2339
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   time       2340 non-null   datetime64[ns]
 1   x          2340 non-null   int64         
 2   y          2340 non-null   int64         
 3   direction  2340 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 73.2+ KB


In [6]:
train.describe(include='all').T

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
time,848835,13059,1991-04-01 00:00:00,65,1991-04-01,1991-09-30 11:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
x,848835.0,NaN,NaN,NaN,NaT,NaT,1.138462,0.801478,0.0,0.0,1.0,2.0,2.0
y,848835.0,NaN,NaN,NaN,NaT,NaT,1.630769,1.089379,0.0,1.0,2.0,3.0,3.0
direction,848835,8,EB,156708,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
congestion,848835.0,NaN,NaN,NaN,NaT,NaT,47.815305,16.799392,0.0,35.0,47.0,60.0,100.0


In [7]:
test.describe(include='all').T

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
time,2340,36,1991-09-30 12:00:00,65,1991-09-30 12:00:00,1991-09-30 23:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
x,2340.0,NaN,NaN,NaN,NaT,NaT,1.138462,0.801649,0.0,0.0,1.0,2.0,2.0
y,2340.0,NaN,NaN,NaN,NaT,NaT,1.630769,1.089611,0.0,1.0,2.0,3.0,3.0
direction,2340,8,EB,432,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
